<a href="https://colab.research.google.com/github/TsundereGus/ibd/blob/master/ibd-project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Trabalho Prático 1

In [2]:
import io
import sqlite3
import pandas as pd

In [3]:
!wget --quiet -P /tmp/ https://raw.githubusercontent.com/TsundereGus/ibd/master/despesas.sql

In [4]:
conn = sqlite3.connect('/tmp/consult.db')
cursor = conn.cursor()

f = io.open('/tmp/despesas.sql', 'r', encoding='utf-8')
sql = f.read()
cursor.executescript(sql)

##Questões da Seção 4.1 a serem resolvidas


2. Liste o valor total de despesas públicas por função (nome) e subfunção (nome), apenas
nos casos em que esse valor total excedeu 200 mil reais.


In [ ]:
query = """
SELECT NOME_FUNCAO, NOME_SUBFUNCAO FROM FUNCAO AS F, SUBFUNCAO AS S WHERE F.COD_FUNCAO IN 
(SELECT DET.COD_FUNCAO FROM DETALHAMENTO_FUNCAO AS DET WHERE F.COD_FUNCAO = DET.COD_FUNCAO AND S.COD_SUBFUNCAO = DET.COD_SUBFUNCAO AND DET.COD_DETALHAMENTO IN 
(SELECT DESP.COD_DETALHAMENTO FROM DESPESA_PUBLICA AS DESP WHERE DET.COD_DETALHAMENTO = DESP.COD_DETALHAMENTO AND DESP.VALOR_DESPESA > 200000))
"""

df = pd.read_sql_query(query, conn)
df

,NOME_FUNCAO,NOME_SUBFUNCAO
0,Assistência social,Assistência comunitária
1,Educação,Educação básica


3. Liste todos os nomes dos órgaos e o valor das despesas totais no ano de 2018 para os mesmos, ordenando de forma decrescente pelo valor.

In [5]:
query = """
SELECT O.NOME_ORGAO, SUM(DESP.VALOR_DESPESA) AS DESPESAS_2018 FROM ORGAO AS O, DESPESA_PUBLICA AS DESP WHERE O.COD_ORGAO IN
(SELECT DESP.COD_ORGAO FROM DESPESA_PUBLICA WHERE O.COD_ORGAO = DESP.COD_ORGAO AND ANO_DESPESA = 2018)
GROUP BY O.COD_ORGAO
ORDER BY SUM(DESP.VALOR_DESPESA) DESC
"""

df = pd.read_sql_query(query, conn)
df

,NOME_ORGAO,DESPESAS_2018
0,Ministério da Defesa,353376.70
1,Ministério do Desenvolvimento Social,278586.35
2,"Ministério da Agricultura, Pecuária e Abastec",90836.51
3,Ministério da Educação,64007.82
4,Ministério da Saúde,50971.19
5,Presidência da República,0.00
6,"Ministério da Ciência, Tecnologia, Inovações",0.00
7,Ministério da Fazenda,0.00
8,Ministério da Justiça e Segurança Pública,0.00
9,Ministério das Relações Exteriores,0.00


4. Liste o mês e ano, modalidade, programa de governo e valor de todas as despesas do
órgão Ministério da Fazenda no primeiro semestre de 2018.

In [ ]:
query = """
SELECT MES_DESPESA, ANO_DESPESA, MODALIDADE_DESPESA, NOME_PROGRAMA_GOVERNO, VALOR_DESPESA FROM DESPESA_PUBLICA, PROGRAMA_GOVERNO 
WHERE MES_DESPESA IN (1, 2, 3) AND COD_ORGAO = 25000 ORDER BY VALOR_DESPESA DESC
"""

df = pd.read_sql_query(query, conn)
df

,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,1,2018,Reserva de Contingência,VIVER SEM LIMITE,0
1,2,2018,Reserva de Contingência,VIVER SEM LIMITE,0
2,3,2018,Reserva de Contingência,VIVER SEM LIMITE,0
3,1,2018,Reserva de Contingência,SOS EMERGENCIA,0
4,2,2018,Reserva de Contingência,SOS EMERGENCIA,0
...,...,...,...,...,...
76,2,2018,Reserva de Contingência,PROGRAMA NACIONAL DE MELHORIA DE ACESSO E QUAL...,0
77,3,2018,Reserva de Contingência,PROGRAMA NACIONAL DE MELHORIA DE ACESSO E QUAL...,0
78,1,2018,Reserva de Contingência,MAIS MEDICOS,0
79,2,2018,Reserva de Contingência,MAIS MEDICOS,0


5. Liste o código e nome dos programas de governo que possuiram os 3 maiores valores
de despesas totais na função de educação durante o ano de 2018.

In [ ]:
query = """
SELECT P.COD_PROGRAMA_GOVERNO, NOME_PROGRAMA_GOVERNO FROM PROGRAMA_GOVERNO AS P, DESPESA_PUBLICA AS D WHERE D.COD_DETALHAMENTO IN
(18, 19, 20, 21, 22) AND ANO_DESPESA = 2018 GROUP BY NOME_PROGRAMA_GOVERNO LIMIT 3
"""

df = pd.read_sql_query(query, conn)
df

,COD_PROGRAMA_GOVERNO,NOME_PROGRAMA_GOVERNO
0,24,BRASIL SEM MISERIA
1,18,CADEIAS PUBLICAS
2,8,CIENCIA SEM FRONTEIRAS


8. Liste os nomes das subfunções das despesas públicas envolvidas para cada programa de
governo. Sua consulta deve retornar o nome do programa de governo, a subfunção da
despesa, a quantidade de entradas de despesas públicas e o valor total dessas despesas.

In [ ]:
query = """
SELECT P.NOME_PROGRAMA_GOVERNO, S.NOME_SUBFUNCAO, COUNT(DESP.COD_PROGRAMA_GOVERNO), SUM(DESP.VALOR_DESPESA) FROM PROGRAMA_GOVERNO AS P, SUBFUNCAO AS S, DESPESA_PUBLICA AS DESP WHERE S.COD_SUBFUNCAO IN 
(SELECT DET.COD_SUBFUNCAO FROM DETALHAMENTO_FUNCAO AS DET WHERE S.COD_SUBFUNCAO = DET.COD_SUBFUNCAO AND DET.COD_DETALHAMENTO IN 
(SELECT DESP.COD_DETALHAMENTO FROM DESPESA_PUBLICA WHERE DET.COD_DETALHAMENTO = DESP.COD_DETALHAMENTO)) AND P.COD_PROGRAMA_GOVERNO IN
(SELECT DESP.COD_PROGRAMA_GOVERNO FROM DESPESA_PUBLICA WHERE P.COD_PROGRAMA_GOVERNO = DESP.COD_PROGRAMA_GOVERNO)
GROUP BY P.NOME_PROGRAMA_GOVERNO, DESP.COD_PROGRAMA_GOVERNO, DESP.COD_DETALHAMENTO
"""

#query = """SELECT COD_PROGRAMA_GOVERNO, COD_DETALHAMENTO, SUM(VALOR_DESPESA) FROM DESPESA_PUBLICA WHERE VALOR_DESPESA != 0 GROUP BY COD_PROGRAMA_GOVERNO, COD_DETALHAMENTO ORDER BY SUM(VALOR_DESPESA) DESC"""


df = pd.read_sql_query(query, conn)
df

,NOME_PROGRAMA_GOVERNO,NOME_SUBFUNCAO,COUNT(DESP.COD_PROGRAMA_GOVERNO),SUM(DESP.VALOR_DESPESA)
0,BRASIL SEM MISERIA,Tecnologia da informação,26,58918.28
1,BRASIL SEM MISERIA,Comunicação social,1,0.00
2,BRASIL SEM MISERIA,Assistência comunitária,102,227883.77
3,BRASIL SEM MISERIA,Alimentação e nutrição,112,100373.55
4,BRASIL SEM MISERIA,Educação infantil,2,0.00
...,...,...,...,...
61,VIVER SEM LIMITE,Educação básica,1,0.00
62,VIVER SEM LIMITE,Transferências para a educação básica,2,0.00
63,VIVER SEM LIMITE,Assistência ao portador de deficiência,5,0.00
64,VIVER SEM LIMITE,Desenvolvimento científico,8,0.00


9. Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade,
valor) em que o órgão responsável é aquele que possui o maior número de órgãos
subordinados.

In [84]:
query = """
SELECT DESP.COD_ORGAO, COD_DESPESA, MES_DESPESA, ANO_DESPESA, MODALIDADE_DESPESA, VALOR_DESPESA FROM DESPESA_PUBLICA AS DESP WHERE DESP.COD_ORGAO IN 
(SELECT O.COD_ORGAO FROM ORGAO AS O WHERE DESP.COD_ORGAO = O.COD_ORGAO AND O.QTD_ORGAOS_SUBORDINADOS = 104)
ORDER BY VALOR_DESPESA
"""

query = """SELECT DESP.COD_ORGAO, QTD_ORGAOS_SUBORDINADOS, COD_DESPESA, MES_DESPESA, ANO_DESPESA, MODALIDADE_DESPESA, VALOR_DESPESA FROM DESPESA_PUBLICA AS DESP INNER JOIN
ORGAO AS O ON DESP.COD_ORGAO = O.COD_ORGAO GROUP BY COD_DESPESA HAVING MAX(QTD_ORGAOS_SUBORDINADOS)
"""

df = pd.read_sql_query(query, conn)
df

,COD_ORGAO,QTD_ORGAOS_SUBORDINADOS,COD_DESPESA,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,VALOR_DESPESA
0,20000,2,0,11,2018,Reserva de Contingência,0.0
1,20000,2,1,11,2018,Reserva de Contingência,0.0
2,20000,2,2,12,2018,Reserva de Contingência,0.0
3,20000,2,3,3,2018,Reserva de Contingência,0.0
4,20000,2,4,4,2018,Reserva de Contingência,0.0
...,...,...,...,...,...,...,...
1122,55000,2,1122,9,2018,Reserva de Contingência,0.0
1123,55000,2,1123,10,2018,Reserva de Contingência,0.0
1124,55000,2,1124,11,2018,Reserva de Contingência,0.0
1125,55000,2,1125,12,2018,Reserva de Contingência,0.0


##Questões da Seção 4.2 a serem resolvidas

12

In [87]:
query = """
SELECT NOME_FUNCAO, NOME_SUBFUNCAO FROM FUNCAO AS F, SUBFUNCAO AS S
INNER JOIN DETALHAMENTO_FUNCAO AS DET ON DET.COD_FUNCAO = F.COD_FUNCAO
INNER JOIN DETALHAMENTO_FUNCAO AS DET ON DET.COD_SUBFUNCAO = S.COD_SUBFUNCAO
"""